Python Design Patterns I

In [1]:
#See the files client.py and server.py

In [2]:
# Decorators
# this a tradicional way of making a decorator, 
# but in the following cell is the more pthonic way

import time
def log_calls(func):
    def wrapper(*args, **kwargs):
        now = time.time()
        print("Calling {0} with {1} and {2}".format(func.__name__, args, kwargs))
        return_value = func(*args, **kwargs)
        print("Executed {0} in {1}ms".format(func.__name__, time.time() - now))
        return return_value
    return wrapper

def test1(a, b, c):
    print("\ttest1 called")

def test2(a, b):
    print("\ttest2 called")
    
def test3(a, b):
    print("\ttest3 called")
    time.sleep(1)
    
    
test1 = log_calls(test1)    
test2 = log_calls(test2)
test3 = log_calls(test3)

test1(1,2,3)
test2(4, b=5)
test3(6,7)


Calling test1 with (1, 2, 3) and {}
	test1 called
Executed test1 in 0.0019996166229248047ms
Calling test2 with (4,) and {'b': 5}
	test2 called
Executed test2 in 0.0019998550415039062ms
Calling test3 with (6, 7) and {}
	test3 called
Executed test3 in 1.0248777866363525ms


In [3]:
# another more pythonic way to do it
@log_calls
def test1(a,b,c):
    print("\ttest1 called")

# this way we can decorate more easily but only to functions we define.
# in Case we need to decorate functions that we don't have access like
# 3rd party code, wwe can use the previous cell way to decorate.


# the observer pattern

In [4]:
# several observer objects can inspect if a core value of another object has change.
# the core class will inform all of the attached observers using the method update() 
# that a value has changed. Then the observers can do whatever they want.
# example:
class Inventory:
    def __init__(self):
        self.observers = []
        self._product = None
        self._quantity = 0
    
    def attach(self, observer):
        self.observers.append(observer)
    
    @property
    def product(self):
        return self._product
    @product.setter
    def product(self, value):
        self._product = value
        self._update_observers()
    
    @property
    def quantity(self):
        return self._quantity
    @quantity.setter
    def quantity(self, value):
        self._quantity = value
        self._update_observers()
    
    def _update_observers(self):
        for observer in self.observers:
            observer() # we call the object directly, 
            # this object will have to implement __call__ to process the update.
            
        

In [5]:
# now lets implement a simple observer object
class ConsoleObserver:
    def __init__(self, inventory):
        self.inventory = inventory
        
    def __call__(self):
        print(self.inventory.product)
        print(self.inventory.quantity)
        

In [6]:
i = Inventory()
c = ConsoleObserver(i)
i.attach(c)

In [7]:
i.product = "Widget"

Widget
0


In [8]:
i.quantity = 5

Widget
5


In [9]:
# add an addittional observer
i = Inventory()
c1 = ConsoleObserver(i)
c2 = ConsoleObserver(i)

In [10]:
i.attach(c1)

In [11]:
i.attach(c2)

In [12]:
i.product = "Gadget"

Gadget
0
Gadget
0


# The strategy pattern

In [4]:
# the example is an application that represents a wallpaper
# into a Screen. the image if it is not of the same size
# as the screen, it can get streched, filled with tiles,
# centerded in the screen and so on.
# The application must select a strategy to deal with the image

from PIL import Image

In [7]:
class TiledStrategy:
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = Image.new('RGB', desktop_size)
        num_tiles = [o // i+1 for o, i in zip(out_img.size, in_img.size)]
        for x in range(num_tiles[0]):
            for y in range(num_tiles[1]):
                out_img.paste(in_img,
                                (
                                in_img.size[0] * x,
                                in_img.size[1] * y,
                                in_img.size[0] * (x+1),
                                in_img.size[1] * (y+1)
                                )
                             )
        return out_img
    
class CenteredStrategy:
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = Image.new('RGB',desktop_size)
        left = (out_img.size[0] - in_img.size[0]) // 2
        top = (out_img.size[1] - in_img.size[1]) // 2
        out_img.paste(
            in_img,
            (
                left, top, left+in_img.size[0], top+in_img.size[1]
            )
        )
        return out_img

class ScaledStrategy:
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = in_img.resize(desktop_size)
        return out_img
    

In [8]:
#above is not usual to see in Python, usually with the Strategy approavh
# people create a callable object.
# see below the modification of above code with __call__
class TiledStrategy:
    def __call__(self, img_file, desktop_size):
        self.make_background(img_file, desktop_size)
        
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = Image.new('RGB', desktop_size)
        num_tiles = [o // i+1 for o, i in zip(out_img.size, in_img.size)]
        for x in range(num_tiles[0]):
            for y in range(num_tiles[1]):
                out_img.paste(in_img,
                                (
                                in_img.size[0] * x,
                                in_img.size[1] * y,
                                in_img.size[0] * (x+1),
                                in_img.size[1] * (y+1)
                                )
                             )
        return out_img
    
class CenteredStrategy:
    def __call__(self, img_file, desktop_size):
        self.make_background(img_file, desktop_size)
        
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = Image.new('RGB',desktop_size)
        left = (out_img.size[0] - in_img.size[0]) // 2
        top = (out_img.size[1] - in_img.size[1]) // 2
        out_img.paste(
            in_img,
            (
                left, top, left+in_img.size[0], top+in_img.size[1]
            )
        )
        return out_img

class ScaledStrategy:
    def __call__(self, img_file, desktop_size):
        self.make_background(img_file, desktop_size)
        
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = in_img.resize(desktop_size)
        return out_img
    

In [11]:
test_img = Image.new('RGB',[1024,768])

In [1]:
new_size = [1920, 1080]
desktoper = ScaledStrategy()
desktoper(test_img, new_size)

NameError: name 'ScaledStrategy' is not defined

#    # The state pattern

In [92]:
text = """<book>
<author>Dusty Phillips</author>   
    <publisher>Packt Publishing</publisher>   
    <title>Python 3 Object Oriented Programming</title>   
    <content>    
        <chapter>     
        <number>1</number>    
        <title>Object Oriented Design</title>     
        </chapter>      
        <chapter>     
        <number>2</number>      
        <title>Objects In Python</title>    
        </chapter>    
        </content> 
</book>"""

In [114]:
class Node:
    def __init__(self, tag_name, parent=None):
        self.parent = parent
        self.tag_name = tag_name
        self.children = []
        self.text = ""
    
    def __str__(self):
        if self.text:
            return self.tag_name + ": " + self.text
        else:
            return self.tag_name    

In [115]:
class Parser:
    def __init__(self, parse_string):
        self.max_level = 0
        self.parse_string = parse_string
        self.root = None
        self.current_node = None
        self.state = FirstTag()
        
    def process(self, remaining_string):
        self.max_level += 1
        assert self.max_level < 100
        print(type(self.state))
        remaining = self.state.process(remaining_string, self)
        print(type(self.state))
        print("""remaining: {} 
            ---------------------------
            """.format(remaining))
        if remaining:
            self.process(remaining)
        
    def start(self):
        self.process(self.parse_string)  

In [116]:
class FirstTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        i_end_tag = remaining_string.find('>')
        tag_name = remaining_string[i_start_tag+1:i_end_tag]
        print('tag_name: {}'.format(tag_name))
        root = Node(tag_name)
        parser.root = parser.current_node = root
        parser.state = ChildNode()
        return remaining_string[i_end_tag+1:]       

In [117]:
class ChildNode:
    def process(self, remaining_string, parser):
        stripped = remaining_string.strip()
        if stripped.startswith("</"):
            parser.state = CloseTag()
        elif stripped.startswith("<"):
            parser.state = OpenTag()
        else:
            parser.state = TextNode()
        return stripped

In [118]:
class OpenTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        i_end_tag = remaining_string.find('>')
        tag_name = remaining_string[i_start_tag+1:i_end_tag]
        node = Node(tag_name, parser.current_node)
        parser.current_node.children.append(node)
        parser.current_node = node
        parser.state = ChildNode()
        return remaining_string[i_end_tag+1:]

In [119]:
class CloseTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        i_end_tag = remaining_string.find('>')
        assert remaining_string[i_start_tag+1] == "/"
        tag_name = remaining_string[i_start_tag+2:i_end_tag]
        assert tag_name == parser.current_node.tag_name
        parser.current_node = parser.current_node.parent
        parser.state = ChildNode()
        return remaining_string[i_end_tag+1:].strip()

In [120]:
class TextNode:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        text = remaining_string[:i_start_tag]
        parser.current_node.text = text
        parser.state = ChildNode()
        return remaining_string[i_start_tag:]
    

In [121]:
contents = text
p = Parser(contents)
p.start()

nodes = [p.root]
while nodes:
    node = nodes.pop(0)
    print(node)
    nodes = node.children + nodes
    

<class '__main__.FirstTag'>
tag_name: book
<class '__main__.ChildNode'>
remaining: 
<author>Dusty Phillips</author>   
    <publisher>Packt Publishing</publisher>   
    <title>Python 3 Object Oriented Programming</title>   
    <content>    
        <chapter>     
        <number>1</number>    
        <title>Object Oriented Design</title>     
        </chapter>      
        <chapter>     
        <number>2</number>      
        <title>Objects In Python</title>    
        </chapter>    
        </content> 
</book> 
            ---------------------------
            
<class '__main__.ChildNode'>
<class '__main__.OpenTag'>
remaining: <author>Dusty Phillips</author>   
    <publisher>Packt Publishing</publisher>   
    <title>Python 3 Object Oriented Programming</title>   
    <content>    
        <chapter>     
        <number>1</number>    
        <title>Object Oriented Design</title>     
        </chapter>      
        <chapter>     
        <number>2</number>      
        <

# The singleton pattern

In [2]:
# in this patter only one instance of an object is created. In python there a re no private constructors, so we will ise __new__
# method that wil create an instance only 1 time in the code.

class OneOnly:
    _singleton = None
    def __new__(cls, *args, **kwargs):
        if not cls._singleton:
            cls._singleton = super(OneOnly, cls).__new__(cls, *args, **kwargs)
        return cls._singleton
# wvery time we create a new instance of this calss, we get the same isntance.
o1 = OneOnly()
o2 = OneOnly()
print(o1 == o2)
print(o1)
print(o2)

True


In [3]:
# use module-level variables instead of a singleton
# we can create variables in a module level and access them instead of instances of an object.
# so it shares the same variables across the entire module.
text = """<book>
<author>Dusty Phillips</author>   
    <publisher>Packt Publishing</publisher>   
    <title>Python 3 Object Oriented Programming</title>   
    <content>    
        <chapter>     
        <number>1</number>    
        <title>Object Oriented Design</title>     
        </chapter>      
        <chapter>     
        <number>2</number>      
        <title>Objects In Python</title>    
        </chapter>    
        </content> 
</book>"""

class Node:
    def __init__(self, tag_name, parent=None):
        self.parent = parent
        self.tag_name = tag_name
        self.children = []
        self.text = ""
    
    def __str__(self):
        if self.text:
            return self.tag_name + ": " + self.text
        else:
            return self.tag_name    

class Parser:
    def __init__(self, parse_string):
        self.max_level = 0
        self.parse_string = parse_string
        self.root = None
        self.current_node = None
        self.state = first_tag
        
    def process(self, remaining_string):
        self.max_level += 1
        assert self.max_level < 100
        print(type(self.state))
        remaining = self.state.process(remaining_string, self)
        print(type(self.state))
        print("""remaining: {} 
            ---------------------------
            """.format(remaining))
        if remaining:
            self.process(remaining)
        
    def start(self):
        self.process(self.parse_string)  

class FirstTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        i_end_tag = remaining_string.find('>')
        tag_name = remaining_string[i_start_tag+1:i_end_tag]
        print('tag_name: {}'.format(tag_name))
        root = Node(tag_name)
        parser.root = parser.current_node = root
        parser.state = child_node
        return remaining_string[i_end_tag+1:]       

class ChildNode:
    def process(self, remaining_string, parser):
        stripped = remaining_string.strip()
        if stripped.startswith("</"):
            parser.state = close_tag
        elif stripped.startswith("<"):
            parser.state = open_tag
        else:
            parser.state = text_node
        return stripped

class OpenTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        i_end_tag = remaining_string.find('>')
        tag_name = remaining_string[i_start_tag+1:i_end_tag]
        node = Node(tag_name, parser.current_node)
        parser.current_node.children.append(node)
        parser.current_node = node
        parser.state = child_node
        return remaining_string[i_end_tag+1:]


class CloseTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        i_end_tag = remaining_string.find('>')
        assert remaining_string[i_start_tag+1] == "/"
        tag_name = remaining_string[i_start_tag+2:i_end_tag]
        assert tag_name == parser.current_node.tag_name
        parser.current_node = parser.current_node.parent
        parser.state = child_node
        return remaining_string[i_end_tag+1:].strip()

class TextNode:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        text = remaining_string[:i_start_tag]
        parser.current_node.text = text
        parser.state = child_node
        return remaining_string[i_start_tag:]
    
first_tag = FirstTag()
child_node = ChildNode()
text_node = TextNode()
open_tag = OpenTag()
close_tag = CloseTag()

contents = text
p = Parser(contents)
p.start()

nodes = [p.root]
while nodes:
    node = nodes.pop(0)
    print(node)
    nodes = node.children + nodes
    


<class '__main__.FirstTag'>
tag_name: book
<class '__main__.ChildNode'>
remaining: 
<author>Dusty Phillips</author>   
    <publisher>Packt Publishing</publisher>   
    <title>Python 3 Object Oriented Programming</title>   
    <content>    
        <chapter>     
        <number>1</number>    
        <title>Object Oriented Design</title>     
        </chapter>      
        <chapter>     
        <number>2</number>      
        <title>Objects In Python</title>    
        </chapter>    
        </content> 
</book> 
            ---------------------------
            
<class '__main__.ChildNode'>
<class '__main__.OpenTag'>
remaining: <author>Dusty Phillips</author>   
    <publisher>Packt Publishing</publisher>   
    <title>Python 3 Object Oriented Programming</title>   
    <content>    
        <chapter>     
        <number>1</number>    
        <title>Object Oriented Design</title>     
        </chapter>      
        <chapter>     
        <number>2</number>      
        <

# The template pattern

Let's create a car sales reporter as an example. We can store records of sales in an
SQLite database table.
We have two common tasks we need to perform:
• Select all sales of new vehicles and output them to the screen in a
comma-delimited format
• Output a comma-delimited list of all salespeople with their gross sales
and save it to a file that can be imported to a spreadsheet
These seem like quite different tasks, but they have some common features. In both
cases, we need to perform the following steps:
1. Connect to the database.
2. Construct a query for new vehicles or gross sales.
3. Issue the query.
4. Format the results into a comma-delimited string.
5. Output the data to a file or e-mail.

The query construction and output steps are different for the two tasks, but the
remaining steps are identical. We can use the template pattern to put the common
steps in a base class, and the varying steps in two subclasses.

In [4]:
import sqlite3

conn = sqlite3.connect("chapter10/sales.db")
conn.execute("CREATE TABLE Sales (salesperson text, "
"amt currency, year integer, model text, new boolean)")
conn.execute("INSERT INTO Sales values"
" ('Tim', 16000, 2010, 'Honda Fit', 'true')")
conn.execute("INSERT INTO Sales values"
" ('Tim', 9000, 2006, 'Ford Focus', 'false')")
conn.execute("INSERT INTO Sales values"
" ('Gayle', 8000, 2004, 'Dodge Neon', 'false')")
conn.execute("INSERT INTO Sales values"
" ('Gayle', 28000, 2009, 'Ford Mustang', 'true')")
conn.execute("INSERT INTO Sales values"
" ('Gayle', 50000, 2010, 'Lincoln Navigator', 'true')")
conn.execute("INSERT INTO Sales values"
" ('Don', 20000, 2008, 'Toyota Prius', 'false')")
conn.commit()
conn.close()



In [11]:
class QueryTemplate:
    def connect(self):
        self.conn = sqlite3.connect("chapter10/sales.db")
    def construct_query(self):
        raise NotImplementedError()
    def do_query(self):
        results = self.conn.execute(self.query)
        self.results = results.fetchall()
    def format_results(self):
        output = []
        for row in self.results:
            row =[str(i) for i in row]
            output.append(", ".join(row))
        self.formatted_results = "\n".join(output)
    def output_results(self):
        raise NotImplementedError()
    def process_format(self):
        self.connect()
        self.construct_query()
        self.do_query()
        self.format_results()
        self.output_results()


In [12]:
import datetime

class NewVehicleQuery(QueryTemplate):
    def construct_query(self):
        self.query = "select * from Sales where new='True'"
    
    def output_results(self):
        print(self.formatted_results)

class UserGrossQuery(QueryTemplate):
    def construct_query(self):
        self.query = ("select salesperson, sum(amt) from Sales group by salesperson")
        
    def output_results(self):
        filename = "gross_sales{0}".format(datetime.date.today.strftime("%Y%m%d"))
        with open(filename, 'w') as outfile:
            outfile.write(self.formatted_results)
    